In [1]:
using SDDP, GLPK, JuMP, Pipe, DataFrames, CSV

In [126]:
STAGES = 20

stock_projections = reduce(vcat, [vec(0.98:0.01:1.02).^(i/2) for i in 1:STAGES]')
N1, N2 = size(stock_projections)
probabilities = zeros(size(stock_projections))
for n1 in 1:N1, n2 in 1:N2
    probabilities[n1, n2] = 1/N2
end

In [127]:
stock_projections

20×5 Matrix{Float64}:
 0.989949  0.994987  1.0  1.00499  1.00995
 0.98      0.99      1.0  1.01     1.02
 0.970151  0.985038  1.0  1.01504  1.03015
 0.9604    0.9801    1.0  1.0201   1.0404
 0.950747  0.975187  1.0  1.02519  1.05075
 0.941192  0.970299  1.0  1.0303   1.06121
 0.931733  0.965435  1.0  1.03544  1.07177
 0.922368  0.960596  1.0  1.0406   1.08243
 0.913098  0.955781  1.0  1.04579  1.0932
 0.903921  0.95099   1.0  1.05101  1.10408
 0.894836  0.946223  1.0  1.05625  1.11507
 0.885842  0.94148   1.0  1.06152  1.12616
 0.876939  0.936761  1.0  1.06681  1.13737
 0.868126  0.932065  1.0  1.07214  1.14869
 0.8594    0.927393  1.0  1.07748  1.16012
 0.850763  0.922745  1.0  1.08286  1.17166
 0.842212  0.918119  1.0  1.08826  1.18332
 0.833748  0.913517  1.0  1.09369  1.19509
 0.825368  0.908938  1.0  1.09914  1.20698
 0.817073  0.904382  1.0  1.10462  1.21899

In [128]:
model = SDDP.LinearPolicyGraph(
    stages = STAGES,
    sense = :Max,
    upper_bound = STAGES*1250,
    optimizer = GLPK.Optimizer,
) do subproblem, node

    M = 10000
    c = 1.01
    p = 0.99
    discount = 1/(1.02^node)
    
    stock_val_dist = stock_projections[node, :]
    stock_pr_dist = probabilities[node, :]
    
    
    
    
    @variable(
        subproblem,
        invest >= 0
    )
    
    @variable(
        subproblem,
        sell >= 0
    )
    
    @variable(
        subproblem,
        buy_order,
        Bin,
        SDDP.State,
        initial_value = 0
    )
    
    @variable(
        subproblem,
        sell_order,
        Bin,
        SDDP.State,
        initial_value = 0
    )
    
    @variable(
        subproblem,
        stock_owned,
        SDDP.State,
        initial_value = 0
    )
    
    @variable(
        subproblem,
        cash,
        SDDP.State,
        initial_value = 1000
    )
    
    @variable(
        subproblem,
        wealth,
        SDDP.State,
        initial_value = 0
    )
    
    @constraint(
        subproblem,
        invest_limit,
        invest <= cash.in
    )
    
    @constraint(
        subproblem,
        buy_order_update,
        invest <= M * buy_order.in
    )
    
    @constraint(
        subproblem,
        buy_order_forced,
        invest >= cash.in - M * (1 - buy_order.in)
    )
    
    @constraint(
        subproblem,
        sell_limit,
        sell <= stock_owned.in 
    )
    
    @constraint(
        subproblem,
        stock_profit_limit,
        sell <= M * sell_order.in
    )
    
    @constraint(
        subproblem,
        update_stock_owned,
        1.0 * stock_owned.out == stock_owned.in + invest/c - sell
    )


    @variable(
        subproblem,
        stock_return
    )
    
    @variable(
        subproblem,
        stock_return_cumulative,
        SDDP.State,
        initial_value = 1
    )
    
    @constraint(
        subproblem,
        update_returns,
        -stock_return_cumulative.out + 1.0 * stock_return_cumulative.in == 0
    )
    
    SDDP.parameterize(subproblem, stock_val_dist, stock_pr_dist) do ω
    
        JuMP.set_normalized_coefficient(
            update_stock_owned,
            subproblem[:stock_owned].out,
            1/ω
        )
        
        JuMP.set_normalized_coefficient(
            update_returns,
            subproblem[:stock_return_cumulative].in,
            ω
        )
        
        JuMP.fix(stock_return, ω)

    end
    
    
    @constraint(
        subproblem,
        update_cash,
        cash.out == cash.in - invest + p * sell
    )
    
    if node == STAGES-1
       @constraint(
            subproblem,
            buy_order.out == 0
            ) 
        
    end
    
    @constraint(
        subproblem,
        update_wealth,
        wealth.out == discount * cash.out + stock_owned.out
    )
    
    @constraint(
        subproblem,
        buy_or_sell,
        buy_order.out+sell_order.out <= 1
    )
    
    @stageobjective(
        subproblem, 
        wealth.in
    )

end

A policy graph with 20 nodes.
 Node indices: 1, ..., 20


In [129]:
SDDP.train(model; iteration_limit = 100, risk_measure = SDDP.EAVaR(beta = 0.25, lambda = 0.3))

------------------------------------------------------------------------------
                      SDDP.jl (c) Oscar Dowson, 2017-21

Problem
  Nodes           : 20
  State variables : 6
  Scenarios       : 9.53674e+13
  Existing cuts   : false
  Subproblem structure                      : (min, max)
    Variables                               : (16, 16)
    VariableRef in MOI.LessThan{Float64}    : (1, 1)
    VariableRef in MOI.ZeroOne              : (2, 2)
    AffExpr in MOI.LessThan{Float64}        : (5, 5)
    AffExpr in MOI.GreaterThan{Float64}     : (1, 1)
    AffExpr in MOI.EqualTo{Float64}         : (4, 5)
    VariableRef in MOI.GreaterThan{Float64} : (2, 3)
Options
  Solver          : serial mode
  Risk measure    : A convex combination of 0.3 * SDDP.Expectation() + 0.7 * SDDP.AVaR(0.25)
  Sampling scheme : SDDP.InSampleMonteCarlo

Numerical stability report
  Non-zero Matrix range     [7e-01, 1e+04]
  Non-zero Objective range  [1e+00, 1e+00]
  Non-zero Bounds range     [2e+

In [130]:
mylist = [
    :wealth,
    :invest,
    :buy_order,
    :sell_order,
    :sell,
    :stock_owned,
    :cash,
    :stock_return,
    :stock_return_cumulative
]

REPLICATIONS = 100

historical = false
if historical
    simulations = SDDP.simulate(
        model,
        sampling_scheme = SDDP.Historical(
            [(1, 1-6/100), (2, 1 - 2.8/100), (3, 1 + 6/100), (4, 1 - 4.6/100)]
        ),
        REPLICATIONS,
        mylist
    )
else
    simulations = SDDP.simulate(
            model,
            REPLICATIONS,
            mylist
        )
end

show = true
if show
    for j in 1:4, i in 1:STAGES
        println("replication:",j, ", stage:", i)
        for item in mylist
            print(string(item)*"("*string(i)*")=")
            print(simulations[j][i][item])
            print("\n")
        end
        print("\n ----------")
        print("\n ----------\n")
    end
end


replication:1, stage:1
wealth(1)=SDDP.State{Float64}(0.0, 980.3921568627451)
invest(1)=0.0
buy_order(1)=SDDP.State{Float64}(0.0, 0.0)
sell_order(1)=SDDP.State{Float64}(0.0, 0.0)
sell(1)=0.0
stock_owned(1)=SDDP.State{Float64}(0.0, 0.0)
cash(1)=SDDP.State{Float64}(1000.0, 1000.0)
stock_return(1)=1.0099504938362078
stock_return_cumulative(1)=SDDP.State{Float64}(1.0, 1.0099504938362078)

 ----------
 ----------
replication:1, stage:2
wealth(2)=SDDP.State{Float64}(980.3921568627451, 961.1687812379854)
invest(2)=0.0
buy_order(2)=SDDP.State{Float64}(0.0, 0.0)
sell_order(2)=SDDP.State{Float64}(0.0, 0.0)
sell(2)=0.0
stock_owned(2)=SDDP.State{Float64}(0.0, 0.0)
cash(2)=SDDP.State{Float64}(1000.0, 1000.0)
stock_return(2)=0.99
stock_return_cumulative(2)=SDDP.State{Float64}(1.0099504938362078, 0.9998509888978457)

 ----------
 ----------
replication:1, stage:3
wealth(3)=SDDP.State{Float64}(961.1687812379854, 942.3223345470446)
invest(3)=0.0
buy_order(3)=SDDP.State{Float64}(0.0, 0.0)
sell_order(3)=S

In [73]:

init_v_int = 1:(REPLICATIONS * STAGES)
init_v = -1.0 * init_v_int
# stands, replications, stages

df = DataFrame(
    replication = init_v_int,
    stage = init_v_int,
    wealth_in = init_v,
    wealth_out = init_v,
    cash_in = init_v,
    cash_out = init_v,
    stock_owned_in = init_v,
    stock_owned_out = init_v,
    invest = init_v,
    sell = init_v,
    stock_return = init_v,
    stock_return_cumulative_in = init_v,
    stock_return_cumulative_out = init_v
)

#df[:, "total_stages"] .= stages

df[:, "buy_order_in"] .= 0.0
df[:, "buy_order_out"] .= 0.0
df[:, "sell_order_in"] .= 0
df[:, "sell_order_out"] .= 0

row = 1
for replication in 1:REPLICATIONS
    for stage in 1:STAGES
        for item in mylist
            
            df[row, :replication] = replication
            df[row, :stage] = stage
            
            if item == :wealth
                df[row, :wealth_in] = simulations[replication][stage][item].in
                df[row, :wealth_out] = simulations[replication][stage][item].out
            elseif item == :cash
                df[row, :cash_in] = simulations[replication][stage][item].in
                df[row, :cash_out] = simulations[replication][stage][item].out
            elseif item == :stock_owned
                df[row, :stock_owned_in] = simulations[replication][stage][item].in
                df[row, :stock_owned_out] = simulations[replication][stage][item].out
            elseif item == :buy_order
                df[row, :buy_order_in] = simulations[replication][stage][item].in
                df[row, :buy_order_out] = simulations[replication][stage][item].out
            elseif item == :sell_order
                df[row, :sell_order_in] = simulations[replication][stage][item].in
                df[row, :sell_order_out] = simulations[replication][stage][item].out
            elseif item == :stock_return_cumulative
                df[row, :stock_return_cumulative_in] = simulations[replication][stage][item].in
                df[row, :stock_return_cumulative_out] = simulations[replication][stage][item].out
            else
                df[row, item] = simulations[replication][stage][item]
                #[item][stand]
            end

        end
        row = row + 1

    end
end

CSV.write("optimization_model_results/results.csv", df)

"optimization_model_results/results.csv"

In [44]:
2-1.04


0.96

In [45]:
"""
transition_matrix = [STATE_1_PR 1-STATE_1_PR; STATE_1_PR 1-STATE_1_PR]

transition_matrix_vector =
Vector{Matrix{Float64}}(
    vcat(
        [[1.0]', [STATE_1_PR 1-STATE_1_PR]] , 
    [transition_matrix for i in 1:stages] 
        ))
"""


"transition_matrix = [STATE_1_PR 1-STATE_1_PR; STATE_1_PR 1-STATE_1_PR]\n\ntransition_matrix_vector =\nVector{Matrix{Float64}}(\n    vcat(\n        [[1.0]', [STATE_1_PR 1-STATE_1_PR]] , \n    [transition_matrix for i in 1:stages] \n        ))\n"

In [47]:

init_v_int = 1:(REPLICATIONS * STAGES)
init_v = -1.0 * init_v_int
# stands, replications, stages

df = DataFrame(
    replication = init_v_int,
    stage = init_v_int,
    wealth_in = init_v,
    wealth_out = init_v,
    cash_in = init_v,
    cash_out = init_v,
    stock_owned_in = init_v,
    stock_owned_out = init_v,
    invest = init_v,
    sell = init_v,
    stock_return = init_v,
    stock_return_cumulative_in = init_v,
    stock_return_cumulative_out = init_v
)

#df[:, "total_stages"] .= stages

row = 1
for replication in 1:REPLICATIONS
    for stage in 1:STAGES
        for item in mylist
            
            df[row, :replication] = replication
            df[row, :stage] = stage
            
            if item == :wealth
                df[row, :wealth_in] = simulations[replication][stage][item].in
                df[row, :wealth_out] = simulations[replication][stage][item].out
            elseif item == :cash
                df[row, :cash_in] = simulations[replication][stage][item].in
                df[row, :cash_out] = simulations[replication][stage][item].out
            elseif item == :stock_owned
                df[row, :stock_owned_in] = simulations[replication][stage][item].in
                df[row, :stock_owned_out] = simulations[replication][stage][item].out
            elseif item == :stock_return_cumulative
                df[row, :stock_return_cumulative_in] = simulations[replication][stage][item].in
                df[row, :stock_return_cumulative_out] = simulations[replication][stage][item].out
            else
                df[row, item] = simulations[replication][stage][item]
                #[item][stand]
            end

        end
        row = row + 1

    end
end

CSV.write("optimization_model_results/results.csv", df)

"optimization_model_results/results.csv"

In [29]:
df

,replication,stage,wealth_in,wealth_out,cash_in,cash_out,stock_owned_in,stock_owned_out
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1,1000.0,1000.0,1000.0,1000.0,0.0,0.0
2,1,2,1000.0,1010.73,1000.0,0.0,0.0,1010.73
3,1,3,1010.73,1000.62,0.0,1000.62,1010.73,0.0
4,1,4,1000.62,1031.99,1000.62,0.0,0.0,1031.99
5,1,5,1031.99,1021.67,0.0,1021.67,1031.99,0.0
6,2,1,1000.0,1006.6,1000.0,0.0,0.0,1006.6
7,2,2,1006.6,1006.6,0.0,0.0,1006.6,1006.6
8,2,3,1006.6,1034.56,0.0,0.0,1006.6,1034.56
9,2,4,1034.56,1024.22,0.0,1024.22,1034.56,0.0


In [62]:
[ [s; s3] for s in [[s1, s2] for s1 in stock_projections for s2 in stock_projections] for s3 in stock_projections][35][3]

1.25

In [31]:
[[s1,s2,s3] for s1 in stock_projections for s2 in stock_projections for s3 in stock_projections][35][3]

1.25

In [4]:
stock_projection_list = [stock_projections, stock_projections]

answ = []
for stock_projection in stock_projection_list
    
    if length(answ) == 0
        answ = [[s] for s in stock_projection]
    else
        answ = [[s; s_new] for s in answ for s_new in stock_projection] 
    end
    
end

In [7]:
function create_joint_probability_distribution(stock_projection_list)  
    answ = []
    
    for stock_projection in stock_projection_list
        if length(answ) == 0
            answ = [[s] for s in stock_projection]
        else
            answ = [[s; s_new] for s in answ for s_new in stock_projection]
        end
    end
        
    answ
end

create_joint_probability_distribution (generic function with 1 method)

In [9]:
create_joint_probability_distribution([stock_projections,stock_projections .* 1.01])

1225-element Vector{Vector{Float64}}:
 [0.95, 0.9594999999999999]
 [0.95, 0.946875]
 [0.95, 0.9258333333333333]
 [0.95, 0.88375]
 [0.95, 0.7575000000000001]
 [0.95, 0.9763333333333334]
 [0.95, 0.9679166666666668]
 [0.95, 0.9538888888888889]
 [0.95, 0.9258333333333333]
 [0.95, 0.8416666666666667]
 [0.95, 0.9931666666666666]
 [0.95, 0.9889583333333333]
 [0.95, 0.9819444444444444]
 ⋮
 [1.25, 1.0520833333333335]
 [1.25, 1.0941666666666665]
 [1.25, 1.0436666666666667]
 [1.25, 1.0520833333333335]
 [1.25, 1.0661111111111112]
 [1.25, 1.0941666666666665]
 [1.25, 1.1783333333333335]
 [1.25, 1.0605]
 [1.25, 1.073125]
 [1.25, 1.0941666666666665]
 [1.25, 1.13625]
 [1.25, 1.2625]

In [69]:
[[s;s_new] for s in [1] for s_new in stock_projections]

35-element Vector{Vector{Float64}}:
 [1.0, 0.95]
 [1.0, 0.9375]
 [1.0, 0.9166666666666666]
 [1.0, 0.875]
 [1.0, 0.75]
 [1.0, 0.9666666666666667]
 [1.0, 0.9583333333333334]
 [1.0, 0.9444444444444444]
 [1.0, 0.9166666666666666]
 [1.0, 0.8333333333333334]
 [1.0, 0.9833333333333333]
 [1.0, 0.9791666666666666]
 [1.0, 0.9722222222222222]
 ⋮
 [1.0, 1.0416666666666667]
 [1.0, 1.0833333333333333]
 [1.0, 1.0333333333333334]
 [1.0, 1.0416666666666667]
 [1.0, 1.0555555555555556]
 [1.0, 1.0833333333333333]
 [1.0, 1.1666666666666667]
 [1.0, 1.05]
 [1.0, 1.0625]
 [1.0, 1.0833333333333333]
 [1.0, 1.125]
 [1.0, 1.25]